## Starter code for assignment 1
This notebook contains the starter code for assignment 1, with unfinished sections for your to complete. Please work through it, and add your code where instructed. There are short-answer writen questions as well at the bottom of the notebook.

When you're finished, save and submit your completed notebook, including the output from your final run.

In [0]:
import matplotlib.pyplot as plt

# Display figures inline (rather than opening a new window)
%matplotlib inline

import numpy as np

# For compatibility between Python 2 and 3
from six.moves import urllib

from sklearn.model_selection import StratifiedShuffleSplit    

from scipy import stats

## Download the mini-cifar data
Understand the format, and visualize images.

In [0]:
source = "https://storage.googleapis.com/jbgordon/mini-cifar.npz"
dest = "mini-cifar.npz"
urllib.request.urlretrieve(source, dest)
loaded = np.load(open(dest))
examples, labels, class_names = loaded["X"], loaded["y"], loaded["class_names"]

Let's look at the format of an image. Below we see the shape of our data is n_examples x 32 x 32 x 3. The dimensions correspond to:
* Number of examples.
* Rows in an image.
* Columns in an image.
* Color channels (R,G,B).

In [0]:
examples.shape

Let's inspect the labels.

In [0]:
labels.shape

This dataset is balanced (contains an equal number of examples of each class). We've stored these in *class_names* for convenience.

In [0]:
stats.itemfreq(labels)

In [0]:
class_names

Let's display a few examples from each class. 

In [0]:
num_classes = len(class_names)
examples_per_class = 5
# We'll create a grid of plots, then populate them with images
f, ax = plt.subplots(examples_per_class, num_classes)
for class_idx in range(len(class_names)):
    # Find the indicies in the examples for this class
    matching_indices = np.where(labels == class_idx)[0]
    for example_n in range(examples_per_class):
        # The images are stored as floats but need to be converted to ints
        # so they display properly
        example = examples[matching_indices[example_n]].astype('uint8')
        ax[example_n, class_idx].imshow(example)
        ax[example_n, class_idx].axis('off')
        plt.axis('off')
        if example_n == 0:
            ax[example_n, class_idx].set_title(class_names[class_idx])
f.set_size_inches(8,4)
plt.show() 

In case that code is a bit of a handful, here's a block you can use to display a single image.

In [0]:
idx = 0
plt.imshow(examples[idx].astype('uint8'))
plt.title(class_names[labels[idx]])
plt.axis('off')
plt.show()

## Prepare the data

In [0]:
# Flatten the images (reshaping from n_examples x 32 x 32 x 3 to n_examples x 3072)
examples = np.reshape(examples, (examples.shape[0], -1))
examples.shape

Next, we'll divide the data into train and test.

In [0]:
# A helpful utility to create balanced splits
# See: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html
sss = StratifiedShuffleSplit(train_size=1200, n_splits=1, 
                             test_size=200, random_state=0)  

for train_index, test_index in sss.split(examples, labels):
    X_train, X_test = examples[train_index], examples[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [0]:
print("Train examples: %d" % X_train.shape[0])
print("Test examples: %d" % X_test.shape[0])

## Part 1) Complete the kNN classifer
You'll need to modify it to a) compute distance between examples, and b) to work with different values of k.

In [0]:
class KNNClassifier(object):
    def __init__(self):
        pass
    
    def train(self, X_train, y_train):
        """
        Train the model.

        Inputs:
        - X_train: A numpy array of shape (train_examples, features) 
        containing the training data.
        - y_train: A numpy array of shape (train_examples,) 
        containing the training labels.
        """
        self.X_train = X_train
        self.y_train = y_train
    
    def distance_matrix(self, X_test):
        """
        Calculate a distance matrix.

        Inputs:
        - X: A numpy array of shape (test_examples, pixels) containing the test data.

        Returns:
        - distances: A numpy array of shape (test_examples, train_examples). 
        Distances[i, j] gives the Euclidean distance between the ith testing point 
        and the jth training point. 
        """
        test_size = X_test.shape[0]
        train_size = self.X_train.shape[0]
     
        # Rows give the distance from a test example to every training example
        distance_matrix = np.zeros((test_size, train_size)) 
        
        ########################################################################
        # TODO: Modify the code below.
        # 
        # Calculate the Euclidian distance between every ith test example
        # and jth training example, then store the result in the distance
        # matrix.
        # 
        # As implemented, this method currently computes a random distance.
        # Repalce that with Euclidian.
        # 
        # (Aside, this pair of nested for loops woefully inefficient, and 
        # can be improved by using vectorized operations in NumPy. 
        # Runtime performance is not important for this assignment).
        # 
        ########################################################################
        length = self.X_train[0].shape[0] # Get the number of elements in each sample

        for i in range(test_size):
            for j in range(train_size):
                
                # Calculate the distance between X_test[i] and self.X_train[j]
                distance = np.sum((X_test[i]-self.X_train[j])**2)**0.5 # Vectorized calculation
               
                # Store the result in the distance matrix.
                distance_matrix[i,j] = distance

        return distance_matrix
    
    def predict(self, X_test, distance_matrix, k=1):
        """
        Predict labels for the testing data.

        Inputs:
        - X_test: A numpy array of shape (test_examples, features) containing the test data.
        - distance_matrix: A previously computed distance matrix, described above.
        
        Returns:
        - predictions: A numpy array of shape (test_examples,). 
        predictions[i] contains the predicted label for the ith testing example. 
        """
        
        # To make a prediction, we'll find the k-nearest training examples
        # to a test example in our distance matrix, and return their 
        # most common label as our prediction. Ties may be broken randomly.
        
        ########################################################################
        # TODO: Modify the code below.
        #
        # As written, this method works properly for k=1.
        # Improve it so it works for other values of k.
        #
        # **Hints**
        # 
        # We currently use np.argmin to find the index of the element with minimum 
        # distance. See the doc for ```np.argsort``` to do something similar 
        # for multiple elements. 
        # https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html
        # 
        ########################################################################
    
        test_size = X_test.shape[0] # Number of test set elements
        predictions = np.zeros(test_size) # Initialize an empty array for predicted labels
        
        for i in range(test_size):
            row = distance_matrix[i]
            
            # This finds the index of the k-nearest training example.
            sorted_idx = np.argsort(row) # Return the indices that would sort the array
      
            k_nearest_idx = sorted_idx[:k] # Return the indices of the k smallest distances
            
            # Look up the correspond label. 
            tmp_labels = np.zeros(k) # Initialize an empty array to store the labels of the neighbors
            for j in range(k):
              tmp_labels[j] = self.y_train[k_nearest_idx[j]] # Store labels for k-nearest neighbors
              
            label = stats.mode(tmp_labels)[0][0]
                          
            # Store the result
            predictions[i] = label
            
            # Above is our prediction for this testing example. 
            # This works properly for k=1. You'll need to modify it
            # to work with other values of k.
            
        return predictions

In [0]:
classifier = KNNClassifier()
classifier.train(X_train, y_train)
distance_matrix = classifier.distance_matrix(X_test) 
predictions = classifier.predict(X_test, distance_matrix, k=1)

In [0]:
def accuracy(predicted, actual):
    correct = np.sum(predicted == actual)
    total = predicted.shape[0]
    accuracy = float(correct) / total
    return accuracy

acc = accuracy(predictions, y_test)
print('Accuracy: %.2f' % (acc))

if acc > 0.50:
    print ("Congrats! Your classifier is working well on this data.")
else:
    print ("Keep at it.")

## Part 2) Short answer questions

Just a few sentences each is fine. Please write your answers in-line below.

1) Is the kNN classifier invariant to image orientation? E.g., what would happen if we used the model to make a prediction on a picture of a car that happened to upside down (assuming the training set contained images of cars right side up).

KNN classifier is not invariant to image orientation. When we rotate the image (or mirror it), the pixel values are all re-ordered when we flatten pixel matrix into an array. The input features would be very different from normal right side up car features for training. Therefore, when we train the classifier with cars right side up, the classifier would very likely not be able to recognize the car upside down.

2) What are the pros/cons of using raw pixel values as features? It is sensisble to use them when working with images? Why or why not?

If using raw pixel values as features, it will require a CNN and much more computational resources to train the network. If we have a high level of understanding of the problem, it is not sensible to use raw pixels when working with images because you can use feature engineering to solve the problem with less data more elegantly. 

3) Is it important to normalize the data (e.g., by subtracting the mean and dividing by the standard deviation) when working with kNN? Why or why not?

It is important to normalize the data because KNN uses Euclidean distance for classification. For example, in a 2-dimensional dataset, if the range of one dimension is significantly different from the other dimension, the KNN algorithm may produce incorrect classification result.

## Part 3) Collect your own dataset

In this section, you'll write code to:

* Download a small dataset (say, of 5 images - these can be of anything you like) from the web. You can use the urllib library above to download them.

* Next, convert these images into an appropriate format in NumPy.

* Finally, use your completed classifier above to train a model on these images and make predictions. Report your accuracy.

In [0]:
## Your code here.
from PIL import Image
import urllib, cStringIO
urls = ['http://www.bas-uk.com/sites/default/files/suri1_0.jpg', # Alpaca
        'http://www.bas-uk.com/sites/default/files/alpaca2001_1.jpg', # Alpaca
        'https://hivemodern.com/public_resources/vl38-table-lamp-vilhelm-lauritzen-louis-poulsen.jpg', # Lamp
        'https://gabbyhome.com/wp-content/uploads/SCH-153745-150x150.jpg', # Lamp
        'https://www.openherd.com/userPhotos/Large/2015_635666082152776416.jpg'] # Alpaca 

In [0]:
Labels = np.array([1, 1, 0, 0, 1])

In [0]:
data = []
for i in urls:
  f = cStringIO.StringIO(urllib.urlopen(i).read())
  img = Image.open(f)
  img.load()
  data.append(np.asarray(img, dtype = 'int32'))

In [0]:
data = np.array(data)
data.shape

In [0]:
Labels.shape

In [0]:
stats.itemfreq(labels)

In [0]:
Class_names = (['Lamp', 'Alpaca'])

In [0]:
# Flatten the images (reshaping from n_examples x 32 x 32 x 3 to n_examples x 3072)
data = np.reshape(data, (data.shape[0], -1))
data.shape

In [0]:
X_train1 = data[0:4]
X_test1 = data[4:]
y_train1 = Labels[0:4]
y_test1 = Labels[4]

In [0]:
print("Train examples: %d" % X_train1.shape[0])
print("Test examples: %d" % X_test1.shape[0])

In [0]:
classifier = KNNClassifier()
classifier.train(X_train1, y_train1)
distance_matrix = classifier.distance_matrix(X_test1) 
predictions = classifier.predict(X_test1, distance_matrix, k=1)

In [0]:
def accuracy(predicted, actual):
    correct = np.sum(predicted == actual)
    total = predicted.shape[0]
    accuracy = float(correct) / total
    return accuracy

acc = accuracy(predictions, y_test)
print('Accuracy: %.2f' % (acc))

if acc > 0.50:
    print ("Congrats! Your classifier is working well on this data.")
else:
    print ("Keep at it.")

When you're finished, save your notebook (including the output from your final run) and submit your notebook on CourseWorks. Your completed notebook should run end-to-end with no user intervention required.